In [1]:
#Import libraries

import cv2
import mediapipe as mp
from scipy.spatial import distance as dist
import pygame


pygame 2.5.2 (SDL 2.28.3, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Function to calculate eye aspect ratio (EAR)

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [3]:
# Function to play warning sound

def play_warning_sound():
    pygame.mixer.init()
    warning_sound=pygame.mixer.Sound(r'D:\DL projects\OPENCV_PROJECTS\emergency-alarm-with-reverb-29431.mp3')
    warning_sound.play()


#Function to stop warning sound
    
def stop_warning_sound():
    pygame.mixer.stop()
    

In [4]:
# Initialize MediaPipe Face Mesh

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

In [5]:
# Custom indices for left and right eyes

left_eye_indices = [362, 385, 387, 263, 373, 380]
right_eye_indices = [33, 160, 158, 133, 153, 144]

In [6]:
min_ear=0.25
max_frame_count=6
eye_closed_counter=0

warning_playing=False

In [7]:
video= cv2.VideoCapture(r'stock-footage-tired-man-driving-on-highway-eyes-closes-drop-and-falls-asleep-behind-wheel-loses-focus-suddenly.mp4')
fps=int(video.get(cv2.CAP_PROP_FPS))

while True:
    success, frame = video.read()
    if not success:
        break
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            
            # Extract landmarks for left and right eyes using custom indices
            landmarks = [((int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0]))) for landmark in face_landmarks.landmark]

            # Extract landmarks for left eye
            left_eye_landmarks = [landmarks[i] for i in left_eye_indices]

            # Extract landmarks for right eye
            right_eye_landmarks = [landmarks[i] for i in right_eye_indices]

            # Draw landmarks for left and right eyes
            for landmark in left_eye_landmarks:
                cv2.circle(frame, landmark, 2, (0, 255, 0), -1)

            for landmark in right_eye_landmarks:
                cv2.circle(frame, landmark, 2, (0, 255, 0), -1)

            # Calculate Eye Aspect Ratio (EAR) for left and right eyes
            ear_left = eye_aspect_ratio(left_eye_landmarks)
            ear_right = eye_aspect_ratio(right_eye_landmarks)

            # Average eye aspect ratio for both eyes
            ear_avg = (ear_left + ear_right) / 2.0

            #Check for drowesiness
            if ear_avg < min_ear:
                eye_closed_counter+=1
            else:
                eye_closed_counter=0

            # Display EAR on the frame
            cv2.putText(frame, "EAR: {:.2f}".format(ear_avg) ,(300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            
            if eye_closed_counter >= max_frame_count:
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
               
                if not warning_playing:
                    play_warning_sound()
                    warning_playing=True

            else:
                    if warning_playing:
                        stop_warning_sound()
                        warning_playing=False


    # Display the frame
    cv2.imshow('Drowsiness Detection', frame)
    if cv2.waitKey(1000//fps) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
video.release()
cv2.destroyAllWindows()